In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os 
import scipy.stats as stats
plt.style.use('ggplot')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import GridSearchCV

In [3]:
df= pd.read_csv('../LinearRegression/housePredcition/data.csv')
print("null check ",df.isnull().sum())
print("-"*50)
print('info check', df.info())

null check  date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null  

In [4]:
df.sample()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
1183,2014-05-22 00:00:00,431500.0,3.0,3.5,1900,1612,2.0,0,0,3,1430,470,2008,0,6527 41st Ave SW,Seattle,WA 98136,USA


In [5]:
df['date'] = pd.to_datetime(df['date'])
int_cols = ['bedrooms', 'waterfront', 'view', 'condition']
df[int_cols] = df[int_cols].astype(int)

In [6]:
category=df.select_dtypes(include='object').columns
for i in category:
    print (i)
    print('-'*50)
    print(df[i].nunique())

    print(df[i].unique())
    

street
--------------------------------------------------
4525
['18810 Densmore Ave N' '709 W Blaine St' '26206-26214 143rd Ave SE' ...
 '759 Ilwaco Pl NE' '5148 S Creston St' '18717 SE 258th St']
city
--------------------------------------------------
44
['Shoreline' 'Seattle' 'Kent' 'Bellevue' 'Redmond' 'Maple Valley'
 'North Bend' 'Lake Forest Park' 'Sammamish' 'Auburn' 'Des Moines'
 'Bothell' 'Federal Way' 'Kirkland' 'Issaquah' 'Woodinville'
 'Normandy Park' 'Fall City' 'Renton' 'Carnation' 'Snoqualmie' 'Duvall'
 'Burien' 'Covington' 'Inglewood-Finn Hill' 'Kenmore' 'Newcastle'
 'Mercer Island' 'Black Diamond' 'Ravensdale' 'Clyde Hill' 'Algona'
 'Skykomish' 'Tukwila' 'Vashon' 'Yarrow Point' 'SeaTac' 'Medina'
 'Enumclaw' 'Snoqualmie Pass' 'Pacific' 'Beaux Arts Village' 'Preston'
 'Milton']
statezip
--------------------------------------------------
77
['WA 98133' 'WA 98119' 'WA 98042' 'WA 98008' 'WA 98052' 'WA 98115'
 'WA 98038' 'WA 98045' 'WA 98155' 'WA 98105' 'WA 98074' 'WA 98106'


In [7]:
## don't think stateZip will be cruicial so dropping it
## has around 4525 unique street so dropping street columns for now 
## as all has rows has 'usa' country so not important
df.drop(columns=['street', 'statezip','country'],inplace=True)


In [8]:
df.sample()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city
1957,2014-06-05,292000.0,5,2.5,2490,7666,1.0,0,0,4,1490,1000,1989,0,Kent


In [9]:
##now only one category columns left (city) may be need to perform one hot encoding lets see

In [10]:
## removing date for now 
df.drop(columns='date',inplace=True)

In [11]:
## perform one hot encoding on city columns 
df=pd.get_dummies(df, columns=['city'], drop_first=True,dtype=np.int_)

In [68]:
## before using decision Tree lets split label and feature adn perform train test split 
x=df.iloc[:,1:]
y=df['price']

In [69]:
x_train , x_test, y_train , y_test=train_test_split(x, y , test_size=0.2, random_state=42)

In [70]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3680, 55)
(920, 55)
(3680,)
(920,)


In [81]:
param_dist={
    "min_samples_split":[50,55,60,65,70,75,80,85,90,100],
    "max_features":[20,21,22,23,24,25]
}
clf=DecisionTreeRegressor()
clf.fit(x_train,y_train)
grid = GridSearchCV(estimator=clf,param_grid=param_dist, cv=10,n_jobs=-1,scoring="r2")
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.best_params_)



0.5193519946192562
{'max_features': 25, 'min_samples_split': 75}


In [82]:
clf2=DecisionTreeRegressor(max_features= 21, min_samples_split= 65)
clf2.fit(x_train,y_train)
y_pred2=clf2.predict(x_test)
print(r2_score(y_test, y_pred2))

0.03630591251420556
